# Processamento de Dados

Como segunda etapa do projeto, é preciso tratar os dados **antes** de os mesmos serem utilizados em um modelo de aprendizado de máquina. Todas as transformações testadas e aplicadas serão vistas neste Notebook.

## Pacotes e Inicialização Importantes

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import itertools

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# usado para o valor float do Pandas ficar neste formato
pd.options.display.float_format = '{:,.2f}'.format

## Leitura dos dados de treino

In [2]:
train_path = '../data/raw/treino.csv'

df = pd.read_csv(train_path, sep=',')
df.head()

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,1,0.77,45,2,0.80,"9,120.00",13,0,6,0,2.00
1,0,0.96,40,0,0.12,"2,600.00",4,0,0,0,1.00
2,0,0.66,38,1,0.09,"3,042.00",2,1,0,0,0.00
3,0,0.23,30,0,0.04,"3,300.00",5,0,0,0,0.00
4,0,0.91,49,1,0.02,"63,588.00",7,0,1,0,0.00


## Tratamento de valores faltantes

Apenas duas colunas dos dados se encontram com valores faltantes. Como será testado algumas maneiras de formatar tais dados, por enquanto, podemos simplesmente preenche-los com $0$.

In [3]:
df = df.fillna(0)

# Divisão dos Dados

Antes de trabalhar com o processamento dos dados, é necessário fazer duas etapas:

* Organizar os Dados em X e y - É preciso ter separado as Entradas e Saídas do modelo, que neste caso chamaremos de X e y, respectivamente.
* Treino e Teste - Para podermos avaliar se o processamento dos dados teve resultados positivos, será necessário dividir a informação em dois grupos: treino e teste, onde a primeira é usada para treinar os modelos com os dados processados, e a segunda irá avaliar os seus desempenhos em dados não vistos.

## Dividindo em X e y

In [4]:
X = df.drop(columns=['inadimplente'])
y = df['inadimplente']

In [5]:
print('Shape do X:', X.shape) # 10 Features
print('Shape do y:', y.shape) # 1 target

Shape do X: (110000, 10)
Shape do y: (110000,)


## Treino e Teste

Para fazer tal divisão, será utilizado a função `train_test_split` da biblioteca **sklearn**. Os dados serão dividos na seguinte proporção: 80% de Treino e 20% de Teste. Para não ocorrer nenhum vazamento de informação, estes dados só serão avaliados com o **melhor modelo encontrado**. Como a quantidade de dados positivos é baixa, será trabalhado com a técnica de validação cruzada para obter os resultados dos modelos.

**A divisão foi feita utilizando o argumento stratify, que garante que a distribuição dos exemplos continue a mesma entre treino e teste**.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
y_train.value_counts() # Quantidade de dados de treino

0    82135
1     5865
Name: inadimplente, dtype: int64

In [10]:
y_test.value_counts() # Quantidade de dados de Treino

0    20534
1     1466
Name: inadimplente, dtype: int64

# Teste Inicial

Para se ter uma visão inicial dos dados, será feito um treinamento dos dados em 3 modelos diferentes: 

* Árvores de Decisão
* Random Forest
* XGBoost

Neste primeiro teste, será utilizado os dados "crus", apenas modificando os valores nulos para **ZERO**. Com isso, é possível ter uma estimativa base do desempenho desses dados nos modelos e o que precisa ser batido para termos um resultado melhor. Para isso, será utilizado o a função `cross_val_score` que retorna o resultados de uma certa métrica na validação cruzada. Nestes testes será escolhido um total de 7 *Folds* para validação, permitindo que sejam feitos 7 testes com uma divisão próxima de [85% teste e 15% validação]

Os modelos a serem treinados irão utilizar o balanceamento dos pesos, para que todas as classes tenham a mesma importância. Para verificar o desempenho, será avaliado a métrica de *F1-Score*.

Primeiro, será testado uma Árvore de Decisão, pois é um dos modelos mais simples e rápidos de ser treinado, e já é possível ter uma pequena noção do que esperar como resultado. 

In [11]:
dtc = DecisionTreeClassifier(random_state=0, class_weight='balanced')
results = cross_val_score(dtc, X_train, y_train, 
                          cv=7, scoring='f1', n_jobs=-1)

print('Árvore de Decisão')
print('F1-Score médio em 7 folds: {:.3f}'.format(np.mean(results)))
print('Desvio Padrão do F1 em 7 folds: {:.3f}'.format(np.std(results)))
print(results)

Árvore de Decisão
F1-Score médio em 7 folds: 0.255
Desvio Padrão do F1 em 7 folds: 0.009
[0.26161369 0.27010436 0.24317618 0.24479495 0.25197809 0.25684723
 0.25364078]


Em segundo, passaremos pelas Random Forest. Elas funcionam da mesma maneira que as árvores de decisão, mas utilizando várias "árvóres" geradas aleatóriamente. Assim, é possível ter um modelo mais generalista ao problema que (provavelmente) terá um resultado melhor que o anterior.

In [12]:
rfc = RandomForestClassifier(random_state=0, class_weight='balanced')
results = cross_val_score(rfc, X_train, y_train, 
                          cv=7, scoring='f1', n_jobs=-1)

print('Random Forest')
print('F1-Score médio em 5 folds: {:.3f}'.format(np.mean(results)))
print('Desvio Padrão do F1 em 7 folds: {:.3f}'.format(np.std(results)))
print(results)

Random Forest
F1-Score médio em 5 folds: 0.221
Desvio Padrão do F1 em 7 folds: 0.008
[0.21002838 0.21910112 0.2245283  0.21374046 0.22867854 0.23330198
 0.21821632]


In [37]:
y_train.value_counts()[0] / y_train.value_counts()[1]

14.004262574595055

Por fim, será usado o modelo que é o estado da arte em dados tabulares. O XGBoost também trabalha com árvores de decisão, mas utilizando o agrupamento de diversas árvores para obter o melhor resultado possível através da união de seus resultados.

In [40]:
# o balanceamento no XGBoost pode ser calculado
# através da formula: (exemplos negativos) / (exemplos positivos)
n = y_train.value_counts()[0]
p = y_train.value_counts()[1]
weight = n / p

xgb = XGBClassifier(random_state=0, scale_pos_weight=weight)
results = cross_val_score(xgb, X_train, y_train, 
                          cv=7, scoring='f1', n_jobs=-1)

print('XGBoost')
print('F1-Score médio em 5 folds: {:.3f}'.format(np.mean(results)))
print('Desvio Padrão do F1 em 7 folds: {:.3f}'.format(np.std(results)))
print(results)

XGBoost
F1-Score médio em 5 folds: 0.331
Desvio Padrão do F1 em 7 folds: 0.006
[0.32664465 0.32329317 0.32622656 0.34278416 0.33325006 0.33501769
 0.32756867]


# Tratamento das variáveis

Através dos *insights* retirados da Análise feita anteriormente, é proposto algumas modificações a serem feitas nos dados para tentar obter um melhor resultado.

## `idade`

A variável **idade** possuí poucos valores *outliers* e variando de maneira natural. Esses *outliers* são idades como ZERO e as próximas do 100 anos. É possível que não seja preciso fazer nenhuma mudança, mas para tentar contornar tal problema (se houver), será utilizando a inclusão da **FAIXA ETÁRIA ANS**. A faixa etária ANS é dividida da seguinte maneira:

* a) 0 a 18 anos 
* b) 19 a 23 anos
* c) 24 a 28 anos
* d) 29 a 33 anos
* e) 34 a 38 anos
* f) 39 a 43 anos
* g) 44 a 48 anos
* h) 49 a 53 anos
* i) 54 a 58 anos
* j) 59 anos ou mais.

Além de possuir menos valores possíveis, eles ainda contém **ordem**, o que permite que sejam agrupados em valores sequências. Neste caso, de 1 até 10.

In [14]:
faixa_etaria_ans = [18, 23, 28, 
                    33, 38, 43, 
                    48, 53, 58,]

def get_faixa_etaria(idade):
    
    for i, f in enumerate(faixa_etaria_ans, 1):
        if idade <= f:
            return i
    else:
        return i + 1

def add_faixa_etaria(data):
    
    data['faixa_etaria'] = data['idade'].apply(get_faixa_etaria)
    data.drop(columns=['idade'], inplace=True)

## `numero_de_dependentes`

A maioria dos dados possuí poucos dependentes, enquantos poucos valores possuem uma grande quantidade. Neste caso, é proposto o agrupamento desses dados nos seguintes grupos:

* 0 -> Sem dependentes
* 1 -> Um dependente
* 2 -> Dois dependentes
* 3 -> Três ou Quatro dependentes
* 4 -> Cinco ou mais dependentes

Assim como a idade, este grupo possui **ordem**.

In [15]:
grupo_dependentes = [0, 1, 2, 4]

def get_grupo_dependentes(n):
    
    for i, g in enumerate(grupo_dependentes):
        if n <= g:
            return i
    else:
        return i+1

def add_grupo_dependentes(data):
    
    r = data['numero_de_dependentes'].apply(get_grupo_dependentes)
    data['grupo_dependentes'] = r
    
    data.drop(columns=['numero_de_dependentes'], inplace=True)

## `numero_emprestimos_imobiliarios`

A esmagadora maioria dos dados estão entre o intervalo de 0 até 2, as outras quantidades encontradas estão distribuídas em quantidades pequenas. A solução aplicada para esta variável é deixar como o maior limite em 3, onde qualquer valor acima disso é modificado para 3.

In [16]:
def add_grupo_imobiliario(data):
    
    index = (data['numero_emprestimos_imobiliarios'] > 3)
    col = 'numero_emprestimos_imobiliarios'
    
    data.loc[index, col] = 3
    data.rename(columns={
        'numero_emprestimos_imobiliarios': 'grupo_imobiliario'
    }, inplace=True)

## `util_linhas_inseguras`

Como foi mostrado na análise de dados, a partir do número 2 os valores começam a se distribuir em intervalos bem grandes. A solução aplicada para esta coluna é a mesma da anterior, limitar sua quantidade em 2.

In [17]:
def limit_util_linhas_inseguras(data):
    
    index = (data['util_linhas_inseguras'] > 2.0)
    col = 'util_linhas_inseguras'
    
    data.loc[index, col] = 2.0
    data.rename(columns={
        'util_linhas_inseguras': 'util_linhas_inseguras_limitada'
    }, inplace=True)

## `salario_mensal`

Como os salários variam bastante no conjunto de dados, foi decidido agrupar eles em grupos, para não ter que tratar com grandes quantidades e valores tão diferentes. Os grupos decididos foram os seguintes:

| Grupo | Descrição |
| - | :- |
| 0 | Sem salário definido ou 0 |
| 1 | Até 2 Salários Mínimos |
| 2 | Entre 2 e 4 Salários Mínimos |
| 3 | Entre 4 e 10 Salários Mínimos |
| 4 | Entre 10 e 20 Salários Mínimos |
| 5 | Mais de 20 Salários Mínimos |

In [18]:
salario_minimo =  1_100
grupo_salarial = [2, 4, 10, 20]

def get_grupo_salarial(n):
    
    if (n == np.nan) or (n == 0):
        return 0
    
    for i, g in enumerate(grupo_salarial, 1):
        if n <= g * salario_minimo:
            return i
    else:
        return i+1

def add_grupo_salarial(data):
    
    r = data['salario_mensal'].apply(get_grupo_salarial)
    data['grupo_salarial'] = r
    
    data.drop(columns=['salario_mensal'], inplace=True)

## `vezes_passou_X_dias`

Existe 3 variáveis com o mesmo sentido, apenas modificando a quantidade de dias que teve de atraso. Elas são as seguintes:

* **`vezes_passou_de_30_59_dias`**
* **`numero_de_vezes_que_passou_60_89_dias`**
* **`numero_vezes_passou_90_dias`**

A mudança a ser feita é a seguinte: não será mais contado a quantidade de vezes que a pessoa passou, apenas se ela passou alguma vez ou não. Com isso, cada uma das colunas indicaria se atrasou alguma vez em cada um dos intervalos.

In [19]:
def add_atraso_em_meses(data):
    
    columns = [
        'vezes_passou_de_30_59_dias',
        'numero_de_vezes_que_passou_60_89_dias',
        'numero_vezes_passou_90_dias',
    ]
    
    for v, col in enumerate(columns, 1):
        
        index = (data[col] > 0)
        data.loc[index, col] = 1

## `razao_debito`

Razão débito possui um comportamento parecido com `util_linhas_inseguras`, mas tendo uma parcela maior fora do intervalo de 0 até 1. Mesmo assim, será utilizado a mesma técnica que na variável anterior.

In [20]:
def limit_razao_debito(data):
    
    index = (data['razao_debito'] > 2.0)
    col = 'razao_debito'
    
    data.loc[index, col] = 2.0
    data.rename(columns={
        'razao_debito': 'razao_debito_limitada'
    }, inplace=True)

# Avaliando os diferentes tratamentos

Como não sabemos se tais mudanças terão um bom desempenho na solução do problema, uma possível estratégia é o teste de todas as combinações de tratamentos possíveis, para decidir quais entregam os melhores resultados.

In [47]:
# Tratamentos propostos
methods = [
    add_faixa_etaria,
    add_grupo_dependentes,
    limit_util_linhas_inseguras,
    add_grupo_salarial,
    add_atraso_em_meses,
    limit_razao_debito,
]

# Aplica os métodos no dataset passado
def fix_data(X, y, methods):
    X_new = X.copy()
    y_new = y.copy()
    
    for m in methods:
        m(X_new)
    
    y_new = y_new[y_new.index.isin(X_new.index)]
    return X_new, y_new


# Avalia os dados com uma combinação de métodos.
def test_method(X, y, methods):
    
    X_new, y_new = fix_data(X, y, methods)
    xgb = XGBClassifier(random_state=0, scale_pos_weight=weight)
    results = cross_val_score(xgb, X_new, y_new, 
                              cv=7, scoring='f1', n_jobs=-1)
    
    return results
    

# Testa todas as combinações possíveis de métodos
def test_methods_combinations(X, y):
    
    # Adiciona todas combinações
    combinations = []
    for s in range(0, len(methods)+1):
        combinations += [c for c in itertools.combinations(methods, s)]
    
    # Itera sobre todas as combinações
    results = []
    for i, c in enumerate(combinations, 1):
        print('\r[{}/{}]'.format(i, len(combinations)), end='')
        
        # Testa a combinação e recebe os resultados de F1-Score
        c = list(c)
        r = test_method(X, y, c)
        
        # Pega os nomes das funções
        c = [f.__name__ for f in c]
        
        # Registro do teste
        data = {
            'methods': c, # Lista de métodos
            'f1_mean': np.mean(r), # Média dos F1-Score
            'f1_std': np.std(r), # Desvio Padrão do F1-Score
        }
        
        results += [data]
    
    return results

In [48]:
results = test_methods_combinations(X_train, y_train)

[64/64]

Com os resultados adquiridos, é possível ordenar esta informação para termos a melhor combinação possível em primeiro (maior F1-Score)

In [49]:
results.sort(key=lambda x: x['f1_mean'], reverse=True)

E agora, visualizaremos esta informação em formatado de `DataFrame`.

In [53]:
info = pd.DataFrame(results)

info

,methods,f1_mean,f1_std
0,"[add_faixa_etaria, add_grupo_dependentes]",0.33,0.01
1,"[add_faixa_etaria, add_grupo_dependentes, limi...",0.33,0.01
2,[add_faixa_etaria],0.33,0.01
3,"[add_faixa_etaria, add_grupo_dependentes, limi...",0.33,0.01
4,"[add_faixa_etaria, limit_util_linhas_inseguras...",0.33,0.01
...,...,...,...
59,"[add_grupo_salarial, add_atraso_em_meses, limi...",0.33,0.01
60,"[add_faixa_etaria, add_grupo_dependentes, add_...",0.33,0.01
61,"[limit_util_linhas_inseguras, add_grupo_salari...",0.33,0.01
62,"[add_grupo_dependentes, limit_util_linhas_inse...",0.33,0.01


Podemos ver que todas as 64 combinações tiveram a mesma média de F1-Score (se arredondar os dados para duas casas decimais). Apenas por descaso de consciência, veremos como o desvio padrão se distribuí (quanto menor o desvio, menor a variação do resultado).

In [54]:
info['f1_std'].describe()

count   64.00
mean     0.01
std      0.00
min      0.00
25%      0.01
50%      0.01
75%      0.01
max      0.01
Name: f1_std, dtype: float64

Pela distribuição, é possível ver que o menor está como zero. Isso apenas significa que o arredondamento o fez chegar neste valor. Veremos quais métodos foram usados para encontrar tal resultado.

In [57]:
info.sort_values('f1_std', ascending=False)['methods'].values[0]

['add_faixa_etaria',
 'add_grupo_dependentes',
 'limit_util_linhas_inseguras',
 'add_grupo_salarial',
 'limit_razao_debito']

# Salvando os dados para treinamento

Com os melhores métodos encontrados, podemos tratar os dados e salva-los para utilização no Notebook final que fará a predição da probabilidade dos dados.

## Dados Tratados

Primeiramente, será salvado os dados com os tratamentos que obtiveram o menor desvio padrão.

In [73]:
# Métodos que tiveram o menor desvio padrão
best_methods = [
    add_faixa_etaria,
    add_grupo_dependentes,
    limit_util_linhas_inseguras,
    add_grupo_salarial,
    limit_razao_debito,
]

# Dados de Treinamento processado
X_train_processed, y_train_processed = fix_data(
    X_train, y_train, best_methods)

# Dados de Teste processado
X_test_processed, y_test_processed = fix_data(
    X_test, y_test, best_methods)

In [74]:
# Organizando o treinamento em apenas um dataframe
train_processed = X_train_processed.copy()
train_processed['inadimplente'] = y_train_processed.values

# Organizando o teste em apenas um dataframe
test_processed = X_test_processed.copy()
test_processed['inadimplente'] = y_test_processed.values

In [76]:
# Salvando os dados de treino processado
train_processed.to_csv('../data/processed/train_processed.csv', 
                       sep=',', index=False)

# Salvando os dados de teste (Neste caso, será chamado 
# como dev, pra não ter engano com os dados de teste final)
test_processed.to_csv('../data/processed/dev_processed.csv', 
                      sep=',', index=False)

## Dados "Sem Processamento"

Para finalizar, os dados são salvados sem nenhum tratamento apresentado **apenas a adição de ZERO nos dados faltantes**.

In [77]:
# Organizando o treinamento em apenas um dataframe
train = X_train.copy()
train['inadimplente'] = y_train.values

# Organizando o teste em apenas um dataframe
test = X_test.copy()
test['inadimplente'] = y_test.values

In [78]:
# Salvando os dados de treino processado
train.to_csv('../data/processed/train.csv', 
             sep=',', index=False)

# Salvando os dados de teste (Neste caso, será chamado 
# como dev, pra não ter engano com os dados de teste final)
test.to_csv('../data/processed/dev.csv', 
            sep=',', index=False)